
### This task consists in understanding the mechanics of connecting to an API and the possibilities it holds. It is a powerful tool which supplies the user with a reliable and updatated source of a type of data that is usually paid for.


*  The challenge that was proposed here was to connect to the API from the https://www.football-data.org/ website (**Sign up to get a free trial and your own login and entry token**) and try to obtain 2 random statistics of series of events at our choosing. Because of Bayer Leversuken's phenomenal season, I was drawn to the German League (Bundesliga), so commited myself for that league and to find out two events:


1.   Which are the best teams to hold an advantage at half time and win (at their home side).
2.   Which teams are the best at turning the score after losing at half-time (any side, home or away).


*   I'll now post the general code that will be needed for both these tasks. Then, I'll do each task in order, with some few commentaries along the way.


In [1]:
## Firstly, we import the libraries we'll need.

import requests
import json
import pandas as pd
import numpy as np
import tqdm
from pandas import json_normalize

In [ ]:
# Let's connect to the API in order to access all the competitions it can provide. Although I already told we were doing Germany, it is still needed to see which leagues we have access and their names.
# Through the website above, you can request to create an account, in which you'll recieve in your email. You put them down and do a request to connect. The "json.loads" transforms the file into a python object.
# This script won't run if you don't create your free account and insert your own X authentication token.

url = "http://api.football-data.org/v4/competitions"

headers = {'X-Auth-Token': "insert your token here"}

response = requests.request("GET", url, headers=headers)

json.loads(response.text)

# We name the json.loads command "df_competitions", and the json library allows us to transform the information into a Dataframe. We also choose the "competitions" dictionary, that will provide us with less data but more useful
# about them. Running an .info() command tells us that there are 13 available competitions to analyze from and we then choose the "Bundesliga".

df_competitions = json.loads(response.text)

df_competitions = json_normalize(df_competitions["competitions"])

df_competitions.info()

choosen_competition = ["Bundesliga"]

bundesliga = df_competitions[(df_competitions["name"].isin(choosen_competition))]

# Were we'll do a FOR loop, were we order the endpoint to return the id associated with the Bundesliga ID. Since it's just one league, we had no necessity of doing this, since we could just search for the ID in the
# dataframe, but it serves as practice for when dealing  with a larger volume of data.

for competitionid in tqdm.tqdm(bundesliga["id"]):
  url = "http://api.football-data.org/v4/competitions/{}/matches".format(competitionid)

  headers = {'X-Auth-Token': "insert your token here"}

  response = requests.request("GET", url, headers=headers)

  data = json.loads(response.text)

# New dataframe, where we'll call the "matches" dictionary. Now we have all finished and scheduled games of the Bundesliga, and a plethora of information about each one of them. We only want the finished matches, so we
# filter the dataframe for them.

df_bundesliga = json_normalize(data["matches"])

df_bundesliga = df_bundesliga[(df_bundesliga["status"] == "FINISHED")]

# Simplicity's sake, let's remove some columns. Here I do it in a certain way, and later by another one. Always useful to know more!

df_bundesliga = df_bundesliga.drop(df_bundesliga.columns[[4,5,6,7,8,10,11,14,15,16,18,19,21,24,25,26,29,30,31,38]], axis=1)

df_bundesliga.info()






1.   Which are the best teams to hold an advantage at half time and win (at their home side)?




In [ ]:
# For this task, we'll do some data manipulation in order to get only the teams that were playing home, winning at half-time and kept on winning the match.

df_keep_advantage = df_bundesliga[(df_bundesliga["score.winner"] == "HOME_TEAM") & (df_bundesliga["score.halfTime.home"] > df_bundesliga["score.halfTime.away"])\
                                  & (df_bundesliga["score.fullTime.home"] > df_bundesliga["score.fullTime.away"])]

# Let's keep just some of the columns, by yet another way of filtering and removing records from a dataframe. We'll then group the dataframe by home club.

df_keep_advantage = df_keep_advantage[['homeTeam.name','awayTeam.name','score.winner', 'score.fullTime.home', 'score.fullTime.away',	'score.halfTime.home',	'score.halfTime.away']]

df_keep_advantage = df_keep_advantage.groupby("homeTeam.name").count().sort_values(["awayTeam.name"], ascending =False)

df_keep_advantage = df_keep_advantage.reset_index()

# The last way that I used to change the number of columns, where you name the columns you want to keep, not the ones you're throwing away.

df_keep_advantage = df_keep_advantage.iloc[:, [0,1]]
df_keep_advantage

# Yet, this is hardly a challenge, although we already had the data ready. We'll now calculate the % of those wins and join them in the same dataframe.

df_halftime_win = df_bundesliga[(df_bundesliga["score.halfTime.home"] > df_bundesliga["score.halfTime.away"])]

df_halftime_win = df_halftime_win.groupby("homeTeam.name").count()

df_halftime_win = df_halftime_win.reset_index()

# Let's merge both dataframes, keeping the name of the first and keeping only the first columns. Then we calculate the new column, "% of home wins while winning at half-time" and do some data transformation and
# manipulation in order to match data types and keep all the names clear.

df_keep_advantage = pd.merge(df_keep_advantage, df_halftime_win, on="homeTeam.name", how="outer")

df_keep_advantage = df_keep_advantage.iloc[:, [0,2,1]]

df_keep_advantage["% of home wins while winning at half-time"] = (df_keep_advantage["awayTeam.name_x"] / df_keep_advantage["id"])*100

df_keep_advantage["% of home wins while winning at half-time"] = df_keep_advantage["% of home wins while winning at half-time"].round(2)


df_keep_advantage = df_keep_advantage.rename(columns={"homeTeam.name":"Home Team", "id":"Matches winning at half-time", "awayTeam.name_x": "Home wins while winning at half-time"})
df_keep_advantage = df_keep_advantage.fillna(0)
df_keep_advantage = df_keep_advantage.astype({"Home wins while winning at half-time":"int"})
df_keep_advantage = df_keep_advantage.sort_values(["% of home wins while winning at half-time"], ascending = False)
df_keep_advantage

2. Which teams are the best at turning the score after losing at half-time (any side, home or away).


In [ ]:
# For this task, we need to filter the matches where the the Home Team was losing at half-time and ended up winning in the end OR the same process but with the Away Team.

df_remontada = df_bundesliga[((df_bundesliga["score.halfTime.home"] < df_bundesliga["score.halfTime.away"]) & (df_bundesliga["score.fullTime.home"] > df_bundesliga["score.fullTime.away"])) |\
                                ((df_bundesliga["score.halfTime.home"] > df_bundesliga["score.halfTime.away"]) & (df_bundesliga["score.fullTime.home"] < df_bundesliga["score.fullTime.away"]))]

df_remontada = df_remontada[['homeTeam.name','awayTeam.name','score.winner', 'score.fullTime.home', 'score.fullTime.away',	'score.halfTime.home',	'score.halfTime.away']]

# Let's create a new column called "Winner", so we can have in each row the winner of the match. This way we can group the clubs by wins, and not needing to join the Home Team wins and Away Team wins.
# The where() will substitute the values of homeTeam.name for awayTeam.name if the condition "df_remontada["score.winner"] == "HOME_TEAM"" isn't satisfied.

df_remontada["Winner"] = df_remontada["homeTeam.name"].where(df_remontada["score.winner"] == "HOME_TEAM", df_remontada["awayTeam.name"])

# Now we can group it by winners, and sort it however we want.

df_remontada = df_remontada.groupby("Winner").count().sort_values(["awayTeam.name"], ascending= False)

df_remontada = df_remontada.reset_index()

df_remontada = df_remontada.iloc[:, [0,1]]
df_remontada

# As with the previous task, we'll now calculate the % of those wins and join them in the same dataframe. We can't just divide by the total matches, so we'll have to discovered the amount of matches that each team.
# was losing at half-time.

matches_played = df_bundesliga[(df_bundesliga["score.halfTime.home"] < df_bundesliga["score.halfTime.away"]) | (df_bundesliga["score.halfTime.home"] > df_bundesliga["score.halfTime.away"])]

matches_played["losing_halfTime"] = matches_played["homeTeam.name"].where(matches_played["score.halfTime.home"] < matches_played["score.halfTime.away"], matches_played["awayTeam.name"])

matches_played = matches_played.groupby("losing_halfTime").count()

matches_played = matches_played.reset_index()

# Let's just keep two columns, the Team Name and the "losing_halftime"

matches_played = matches_played.iloc[:, [0,1]]
matches_played.columns = ["Team", "Matches losing at half-time"]

# Now we merge both dataframes and create the new "% of Wins after losing at half-time" column.

df_remontada = pd.merge(df_remontada, matches_played, left_on="Winner", right_on="Team", how="outer")

# Criamos uma nova coluna, que nos vai dar o pretendido: % de vitórias após estar a perder ao intervalo. Para ficar melhor, vamos arredondar a 2 casas decimais.

df_remontada["% of Wins after losing at half-time"] = (df_remontada["homeTeam.name"] / df_remontada["Matches losing at half-time"])*100
df_remontada["% of Wins after losing at half-time"] = df_remontada["% of Wins after losing at half-time"].round(2)

df_remontada.info()

# Now what's left is to do is transform and manipulate the data to our liking.

df_remontada = df_remontada.iloc[:, [2,3,1,4]]

df_remontada = df_remontada.rename(columns={"homeTeam.name":"Wins after losing at half-time"})
df_remontada = df_remontada.fillna(0)
df_remontada = df_remontada.astype({"Wins after losing at half-time":"int"})
df_remontada = df_remontada.sort_values(["% of Wins after losing at half-time"], ascending = False)

df_remontada

In [ ]:
# As a bonus, let's do one more thing. Create a league table, considering that each win gives 3 points, a draw gives 1 point to both team and a team that loses doesn't win any point.

# My first doubt here is how to calculate each team's points, since the winner is indicated by HOME_TEAM, AWAY_TEAM or DRAW and not by their name or ID.
# Therefore, my logic was to create small tables, each with the wins and draws, either home or away. Then, in the end, we'll merge them together.
# Here we use the .value_counts() command to count how many times the "homeTeam_name" apperears in the winners of HOME_TEAM.
home_wins = df_bundesliga[df_bundesliga['score.winner'] == 'HOME_TEAM']['homeTeam.name'].value_counts().reset_index()
home_wins.columns = ["Team_name", "Home_wins"]
away_wins = df_bundesliga[df_bundesliga['score.winner'] == 'AWAY_TEAM']['awayTeam.name'].value_counts().reset_index()
away_wins.columns = ["Team_name", "Away_wins"]
draws_home = df_bundesliga[df_bundesliga["score.winner"] == "DRAW"]['homeTeam.name'].value_counts().reset_index()
draws_home.columns = ["Team_name", "Home_draws"]
draws_away = df_bundesliga[df_bundesliga["score.winner"] == "DRAW"]['awayTeam.name'].value_counts().reset_index()
draws_away.columns = ["Team_name", "Away_draws"]

# We'll merge all tables into a new dataframe, joining them by "Team_name". We'll also take the NaN out, since that some clubs didn't obtain certain results (an away draw, for instance).
df_table = pd.merge(pd.merge(pd.merge(home_wins, away_wins, on = "Team_name"), draws_home, on = "Team_name"), draws_away, on = "Team_name", how = "outer")
df_table = df_table.fillna(0)

# Already with the desired table, we create the column we want to obtain, the Points. Each win is worth 3 points, and we'll sum an extra point for each drawn match, and finally obtaining the league table.
# After that we just transform and model the data to be appealing and easier to deal.
df_table["Points"] = df_table['Home_wins']*3 + df_table["Away_wins"]*3 + df_table["Home_draws"] + df_table["Away_draws"]
df_table["Wins"] = df_table['Home_wins'] + df_table["Away_wins"]
df_table["Draws"] = df_table["Home_draws"] + df_table["Away_draws"]
df_table = df_table.iloc[:, [0,6,7,5]]
df_table[["Wins", "Draws", "Points"]] = df_table[["Wins", "Draws", "Points"]].astype(int)
df_table = df_table.sort_values(["Points"], ascending = False)
df_table